# On a des gros pics. Est-ce que ce sont réellement des pics ou des erreurs dans la donnée ? 
- Les comparer avec Validation individuelles sur des données Novembre 2019 - Mars 2020 
    - Théoriquement, on peut cibler quelques stations (Guillotière, Hôtel de Ville- Louis Paredel, Charpennes) sur Novembre 2019 - Mars 2020
    - Load les données Validation, les aggréger 15 et sélectionner les mêmes période
    - Comparer les deux, voir si les plots sont cohérent.
Si on retrouve bien les mêmes choses, c'est à dire que les données sont belle et bien cohérentes : 
- Pour Mars, Avril, Mai peut-on considérer que les données sont bonne, et essayer de voir s'il y a une explication avec NetMob ?
    -  Si c'est le cas, cela justifierait l'utilisation de NetMob pour améliorer la prédiction. Il restera à savoir comment on peut design notre modèle pour améliorer cette prédiction.  
    - Comment le voir ? On peut commencer par une map dynamique (interactive) d'intensité de présence. 
### Prenons l'exemple de la gare Part-Dieu ou la gare Perrache. Leur prédiction de In est très fortement associé à l'arrivée de train. 


## Problèmes de données : 
- Dans pas mal de station (Croix-Rousse, Guillotière, Cordelier):     
    - Manquant 16h puis pic improbable 16h30 le 21/12/2019
- Souvent, Cordelier en 'Métro 15 min' a une donnée manquante dans la journée 
    - Pic le 6, 7 et 8 décembre (fête des lumières). Les deux sont cohérent. Mais si on regarde plus ne détail, **souvent petit décalage de 15min**  ce qui fait des erreurs de 100/200 personnes supplémentaire entrées dans le métro
- 

Import

In [2]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [3]:
import pandas as pd
from datetime import datetime 
from build_inputs.load_data import load_subway_15_min

Init

In [4]:
#start,end = datetime(2019,11,1),datetime(2020,4,1)
start,end = datetime(2019,3,16),datetime(2019,6,1)

time_step_per_hour = 4 

reindex = pd.date_range(start,end,freq = f'{60/time_step_per_hour}min')
print(f'Number of time-slot: {4*24*(end-start).days}')

Number of time-slot: 7392


Load subway_in 15 min

In [6]:
FOLDER_PATH = '../../../Data/keolis_data_2019-2020/'
txt_path = "Métro 15 minutes 2019 2020.txt"
df_metro_funi_2019_2020 = load_subway_15_min(FOLDER_PATH+txt_path)
df_metro = df_metro_funi_2019_2020[(df_metro_funi_2019_2020.datetime >= start)&(df_metro_funi_2019_2020.datetime < end)]

subway_in = pd.pivot_table(df_metro,index = 'datetime',columns = 'Station',values = 'in',aggfunc = 'sum', fill_value = 0).reindex(reindex).fillna(0)
subway_out = pd.pivot_table(df_metro,index = 'datetime',columns = 'Station',values = 'out',aggfunc = 'sum', fill_value = 0).reindex(reindex).fillna(0)
subway_in

Station,Ampère Victor Hugo,Bellecour,Brotteaux,Charpennes,Cordeliers,Croix Paquet,Croix-Rousse,Cuire,Cusset,Debourg,...,Part-Dieu,Perrache,Place Guichard,Place Jean Jaurès,République Villeurbanne,Sans Souci,Saxe - Gambetta,Stade de Gerland,Valmy,Vieux Lyon
2019-03-16 00:00:00,34.0,396.0,37.0,164.0,143.0,6.0,33.0,17.0,24.0,40.0,...,68.0,144.0,26.0,48.0,53.0,55.0,112.0,59.0,71.0,186.0
2019-03-16 00:15:00,40.0,298.0,56.0,164.0,139.0,3.0,46.0,7.0,20.0,46.0,...,77.0,145.0,31.0,32.0,45.0,55.0,159.0,56.0,41.0,179.0
2019-03-16 00:30:00,17.0,258.0,24.0,74.0,100.0,19.0,6.0,0.0,4.0,13.0,...,43.0,55.0,15.0,19.0,4.0,45.0,98.0,4.0,72.0,153.0
2019-03-16 00:45:00,1.0,44.0,3.0,13.0,13.0,1.0,6.0,3.0,2.0,2.0,...,6.0,5.0,3.0,4.0,0.0,13.0,17.0,4.0,3.0,14.0
2019-03-16 01:00:00,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,4.0,4.0,2.0,2.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-31 23:00:00,34.0,262.0,37.0,157.0,155.0,9.0,51.0,14.0,30.0,66.0,...,74.0,261.0,124.0,42.0,24.0,27.0,127.0,52.0,53.0,117.0
2019-05-31 23:15:00,28.0,325.0,37.0,154.0,126.0,13.0,45.0,8.0,28.0,44.0,...,104.0,140.0,66.0,33.0,14.0,45.0,140.0,46.0,57.0,134.0
2019-05-31 23:30:00,26.0,342.0,42.0,132.0,101.0,5.0,49.0,16.0,14.0,58.0,...,135.0,185.0,57.0,23.0,16.0,25.0,198.0,49.0,53.0,142.0
2019-05-31 23:45:00,18.0,399.0,20.0,86.0,107.0,8.0,50.0,9.0,24.0,41.0,...,66.0,124.0,70.0,24.0,33.0,37.0,165.0,44.0,36.0,126.0


Load Validation individuelles 15 min 

In [83]:
df_valid = pd.read_csv('data/subway15min_from_validation_individuelle_nov2019_mar2020.csv',index_col = 0)
df_valid.index.name='daetime'
df_valid.index = pd.to_datetime(df_valid.index)
df_valid

,AMPERE,BELLECOUR,BROTTEAUX,CHARPENNES,CORDELIERS,CROIX PAQUET,CROIX ROUSSE,CUIRE,CUSSET,DEBOURG,...,PERRACHE,PLACE GUICHARD,PLACE JEAN JAURES,REPUBLIQUE,SANS SOUCI,SAXE GAMBETTA,STADE DE GERLAND,VALMY,VAULX-EN-VELIN LA SOIE,VIEUX LYON
daetime,,,,,,,,,,,,,,,,,,,,,
2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,21.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,3.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-31 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# Dictionnaire pour match les station de df_valid avec celles de subway_in
C_subway_in = ['Ampère Victor Hugo', 'Bellecour', 'Brotteaux', 'Charpennes',
       'Cordeliers', 'Croix Paquet', 'Croix-Rousse', 'Cuire', 'Cusset',

       'Debourg', 'Flachet', 'Foch',  'Fourvière',"Gare d'Oullins",                                   
       'Gare de Vaise', 'Gare de Vénissieux', 'Garibaldi', 'Gorge de Loup',

       'Grange Blanche', 'Gratte Ciel', 'Guillotière', 'Hénon',
       'Hôtel de ville - Louis Pradel', 'Jean Macé', 'La soie',

       'Laurent Bonnevay', 'Laënnec', 'Les Minimes - Théâtres Romains',
       'Masséna', 'Mermoz - Pinel', 'Monplaisir Lumière', 'Parilly',

       'Part-Dieu', 'Perrache', 'Place Guichard', 'Place Jean Jaurès',
       'République Villeurbanne', 'Sans Souci', 'Saxe - Gambetta', 'St Just',

       'Stade de Gerland', 'Valmy', 'Vieux Lyon']

C_valid = ['AMPERE', 'BELLECOUR', 'BROTTEAUX', 'CHARPENNES', 
           'CORDELIERS','CROIX PAQUET', 'CROIX ROUSSE', 'CUIRE', 'CUSSET', 

           'DEBOURG', 'FLACHET','FOCH', 'FOURVIERE', 'OULLINS GARE',
           'GARE DE VAISE', 'GARE DE VENISSIEUX', 'GARIBALDI','GORGE DE LOUP', 
           
           'GRANGE BLANCHE', 'GRATTE CIEL', 'GUILLOTIERE','HENON',
           'HOTEL DE VILLE', 'JEAN MACE',  'VAULX-EN-VELIN LA SOIE',

           'LAURENT BONNEVAY','LAENNEC','MINIMES', 
           'MASSENA', 'MERMOZ PINEL', 'MONPLAISIR LUMIERE','PARILLY', 

           'PART DIEU', 'PERRACHE', 'PLACE GUICHARD','PLACE JEAN JAURES', 
           'REPUBLIQUE', 'SANS SOUCI', 'SAXE GAMBETTA','SAINT JUST',
           
           'STADE DE GERLAND', 'VALMY', 'VIEUX LYON']

valid2subway_in = {val:sub_in for val,sub_in in zip(C_valid,C_subway_in)}

In [85]:
df_valid = df_valid.rename(columns = valid2subway_in)
df_valid.head()

,Ampère Victor Hugo,Bellecour,Brotteaux,Charpennes,Cordeliers,Croix Paquet,Croix-Rousse,Cuire,Cusset,Debourg,...,Perrache,Place Guichard,Place Jean Jaurès,République Villeurbanne,Sans Souci,Saxe - Gambetta,Stade de Gerland,Valmy,La soie,Vieux Lyon
daetime,,,,,,,,,,,,,,,,,,,,,
2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,21.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,3.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load GeoDataFrame 

In [98]:
import geopandas as gpd
from shapely.geometry import Point 
from shapely.errors import ShapelyDeprecationWarning
import warnings

ref_subway = pd.read_csv('data/ref_subway.csv')[['MEAN_X','MEAN_Y','COD_TRG','LIB_STA_SIFO']]
with warnings.catch_warnings():
    warnings.simplefilter("ignore", ShapelyDeprecationWarning)
    ref_subway['geometry'] = ref_subway.apply(lambda row : Point(row.MEAN_X,row.MEAN_Y),axis = 1)

ref_subway['LIB_STA_SIFO'] = ref_subway.apply(lambda row : valid2subway_in[row.LIB_STA_SIFO],axis = 1)
ref_subway = ref_subway[['LIB_STA_SIFO','geometry']]
ref_subway = gpd.GeoDataFrame(ref_subway)
ref_subway.crs = 'epsg:4326'
ref_subway.head()

,LIB_STA_SIFO,geometry
0,Ampère Victor Hugo,POINT (4.82921 45.75311)
1,Bellecour,POINT (4.83340 45.75726)
2,Brotteaux,POINT (4.85943 45.76689)
3,Cordeliers,POINT (4.83581 45.76345)
4,Cuire,POINT (4.83250 45.78526)


# Plotting Pour chaque station. Puis dépots sur une Carte Folium 
On n'a interpolé aucune donnée sur subway_in, ce qu'on devrait faire comme vu dans Opening_data.

In [87]:
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from folium import IFrame
import os
import base64

# Bokeh plot
from bokeh.plotting import figure, output_file, save

# Generate matplotlib .png ou Bokeh html, then save them 

In [99]:
# Step 2: Generate plots for each station and save as images
plot_folder = "save/station_plots"
#plot_folder = "save/16_03_2019_31_05_2019/station_plots"
os.makedirs(plot_folder, exist_ok=True)

for column in df_valid.columns:
    fig, ax = plt.subplots()
    fig.suptitle(f"Temporal Series for {column}")

    ax.plot(df_valid[column].index, df_valid[column].values,label='Validation individuelles',alpha = 0.6)
    ax.plot(subway_in[column].index, subway_in[column].values,label='Métro 15 min',alpha=0.6)

    plt.xlabel("Time Slot")
    plt.ylabel("Flow")

    ax.legend('Dataset: ')
    filepath = os.path.join(plot_folder, f"{column}.png")
    plt.savefig(filepath)
    plt.close()


# Step 1: Generate Bokeh plots and save as HTML
bokeh_folder = "save/bokeh_plots"
#bokeh_folder = "save/16_03_2019_31_05_2019/bokeh_plots"
    
os.makedirs(bokeh_folder, exist_ok=True)

for column in df_valid.columns:
    plot = figure(title=f"Temporal Series for {column}", x_axis_type = 'datetime', x_axis_label='Time Slot', y_axis_label='Flow')
    legend_it = []

    plot.line(df_valid[column].index, df_valid[column].values, legend_label=f'Validation individuelles', line_width=2,color = 'blue',line_alpha = 0.6,muted_color='blue', muted_alpha=0.1)
    #plot.circle(df_valid[column].index, df_valid[column].values,color = 'blue',line_alpha = 0.6)

    plot.line(subway_in[column].index, subway_in[column].values, legend_label=f'Métro 15 min', line_width=2,color = 'orange',line_alpha = 0.6,muted_color='orange', muted_alpha=0.1)
    #plot.circle(subway_in[column].index, subway_in[column].values,color = 'orange',line_alpha = 0.4)

    #plot.add_tools(HoverTool())

    #Interactive legend
    plot.legend.location = "top_left"
    plot.legend.click_policy="mute"

    output_file(os.path.join(bokeh_folder, f"{column}.html"))
    save(plot)

## Folium Map avec popup de Plot Matplotlib format png

In [100]:
# Step 3: Create the interactive map with matplotlib plot
m = folium.Map(location=[ref_subway['geometry'].y.mean(), ref_subway['geometry'].x.mean()], zoom_start=12)

for _, row in ref_subway.iterrows():
    station_id = row['LIB_STA_SIFO']
    geom = row['geometry']
    try: 
        plot_path = os.path.join(plot_folder, f"{station_id}.png")
        encoded = base64.b64encode(open(plot_path, 'rb').read()).decode()
        html = '<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded), width=850, height=550)
        popup = folium.Popup(iframe, max_width=2650)
        folium.Marker([geom.y, geom.x], popup=popup, tooltip=f"Station {station_id}").add_to(m)
    except:
        print(f'No plot for {station_id}')

# Save or display the map
m.save('save/station_map.html')
#m.save('save/16_03_2019_31_05_2019/station_map.html')      
# To display in a Jupyter notebook, simply use: display(m)

No plot for Les Minimes - Théâtres Romains
No plot for St Just
No plot for Fourvière


## Folium Map avec Hyperlink to Bokeh Plot

In [101]:
m = folium.Map(location=[ref_subway['geometry'].y.mean(), ref_subway['geometry'].x.mean()], zoom_start=12)

for _, row in ref_subway.iterrows():
    station_id = row['LIB_STA_SIFO']
    geom = row['geometry']
    try:  
        plt_plot_path = os.path.join(plot_folder, f"{station_id}.png")
        bokeh_plot_path = os.path.join(bokeh_folder, f"{station_id}.html")

        # plt popup :
        #encoded = base64.b64encode(open(plt_plot_path, 'rb').read()).decode()
        #img_html = f'<img src="data:image/png;base64,{encoded}" style="width:80%;margin-bottom:15px;">'

        # Create an HTML string that includes a link to the Bokeh plot HTML file
        link_html = f'<b>Station {station_id}</b><br><a href="{bokeh_plot_path}" target="_blank">Click here for detailed plot</a>'

        # Combine html : 
        #combined_html = f'{link_html}{img_html}'
        #iframe = IFrame(combined_html, width=850, height=550)  # Adjust height as needed to accommodate both image and link
        #popup = folium.Popup(iframe, max_width=300)
    
        popup = folium.Popup(link_html, max_width=300)

        # Create Marker 
        folium.Marker([geom.y, geom.x], popup=popup, tooltip=f"Station {station_id}").add_to(m)
    except:
        print(f'No plot for {station_id}')     

# Save or display the map
m.save('station_map_with_bokeh.html')
#m.save('save/16_03_2019_31_05_2019/station_map_with_bokeh.html')    